## Import library

In [1]:
import sys
sys.path.append('../')

from data_provider.data_factory import data_provider
from exp.exp_basic import Exp_Basic
# from models import Informer, Autoformer, Transformer
from utils.tools import EarlyStopping, adjust_learning_rate, visual
from utils.metrics import metric

import numpy as np
import torch
import torch.nn as nn
from torch import optim

import os
import time

import warnings
import matplotlib.pyplot as plt
import numpy as np

warnings.filterwarnings('ignore')

In [34]:
%matplotib inline
%pylab inline

UsageError: Line magic function `%matplotib` not found.


## Config

In [2]:
# python -u run.py \
#   --is_training 1 \
#   --root_path ./dataset/ETT-small/ \
#   --data_path ETTh1.csv \
#   --model_id ETTh1_96_24 \
#   --model Autoformer \
#   --data ETTh1 \
#   --features M \
#   --seq_len 96 \
#   --label_len 48 \
#   --pred_len 24 \
#   --e_layers 2 \
#   --d_layers 1 \
#   --factor 3 \
#   --enc_in 7 \
#   --dec_in 7 \
#   --c_out 7 \
#   --des 'Exp' \
#   --itr 1

## Load Data

In [3]:
from data_provider.data_loader import Dataset_ETT_hour, Dataset_ETT_minute, Dataset_Custom, Dataset_Pred
from torch.utils.data import DataLoader

In [4]:
shuffle_flag = True
drop_last = True
batch_size = 32
freq = 'h'

Data = Dataset_ETT_hour

timeenc = 0

root_path = '../dataset'
data_path = 'ETTh1.csv'
flag='train'
seq_len = 96
label_len = 48
pred_len = 48
features = 'S'
target = 'OT'

In [5]:
Data = Dataset_ETT_hour
dataset = Data(root_path='../dataset',
                data_path='ETTh1.csv',
                flag='train',
                size=[seq_len, label_len, pred_len],
                features=features,
                target=target,
                timeenc=timeenc,
                freq=freq)

data_loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle_flag,
        num_workers=10,
        drop_last=drop_last) 


In [6]:
batch_x, batch_y, batch_x_mark, batch_y_mark = next(iter(data_loader))

In [7]:
print(batch_x.shape)
print(batch_y.shape)
print(batch_x_mark.shape)
print(batch_y_mark.shape)

torch.Size([32, 96, 1])
torch.Size([32, 96, 1])
torch.Size([32, 96, 4])
torch.Size([32, 96, 4])


In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [9]:
batch_x = batch_x.float().to(device)
batch_y = batch_y.float().to(device)
batch_x_mark = batch_x_mark.float().to(device)
batch_y_mark = batch_y_mark.float().to(device)

In [10]:
# decoder input
dec_inp = torch.zeros_like(batch_y[:, -pred_len:, :]).float()
dec_inp = torch.cat([batch_y[:, :label_len, :], dec_inp], dim=1).float().to(device)

In [11]:
dec_inp.shape

torch.Size([32, 96, 1])

## Encoder

In [12]:
x_enc = batch_x 
x_mark_enc = batch_x_mark
x_dec = dec_inp 
x_mark_dec = batch_y_mark 

In [13]:
from layers.Autoformer_EncDec import series_decomp
from layers.Embed import DataEmbedding, DataEmbedding_wo_pos
from layers.Autoformer_EncDec import Encoder, Decoder, EncoderLayer, DecoderLayer, my_Layernorm, series_decomp

from layers.AutoCorrelation import AutoCorrelation, AutoCorrelationLayer


In [14]:
# Decomp
kernel_size = 25
decomp = series_decomp(kernel_size)

mean = torch.mean(x_enc, dim=1).unsqueeze(1).repeat(1, pred_len, 1)
zeros = torch.zeros([x_dec.shape[0], pred_len, x_dec.shape[2]], device=x_enc.device)

seasonal_init, trend_init = decomp(x_enc)

# decoder input
trend_init = torch.cat([trend_init[:, -label_len:, :], mean], dim=1)
seasonal_init = torch.cat([seasonal_init[:, -label_len:, :], zeros], dim=1)

In [15]:
print(trend_init.shape)
print(seasonal_init.shape)

torch.Size([32, 96, 1])
torch.Size([32, 96, 1])


In [16]:
#-- encoding config 
enc_in = 1
dec_in = 1
d_model = 512
embed = 'timeF'
dropout = 0.05

enc_embedding = DataEmbedding_wo_pos(enc_in, d_model, embed, freq,
                                            dropout).to(device)
dec_embedding = DataEmbedding_wo_pos(dec_in, d_model, embed, freq,
                                            dropout).to(device)

# enc
enc_out = enc_embedding(x_enc, x_mark_enc)

# dec
dec_out = dec_embedding(seasonal_init, x_mark_dec)

In [17]:
factor = 1
n_heads = 8
d_ff = 2048
moving_avg = 25
activation = 'gelu'
e_layers = 2

# Encoder
encoder = Encoder(
    [
        EncoderLayer(
            AutoCorrelationLayer(
                AutoCorrelation(False, factor, attention_dropout=dropout,
                                output_attention=False),
                d_model, n_heads),
            d_model,
            d_ff,
            moving_avg=moving_avg,
            dropout=dropout,
            activation=activation
        ) for l in range(e_layers)
    ],
    norm_layer=my_Layernorm(d_model)
).to(device)

In [18]:
enc_out, attns = encoder(enc_out, attn_mask=None)

In [19]:
enc_out.shape

torch.Size([32, 96, 512])

## Decoder
- 학습 이후에 Autoformer.py 수정해서 prediction plot 그리기 

In [20]:
c_out = 1
d_layers = 1

In [21]:
# Decoder
decoder = Decoder(
    [
        DecoderLayer(
            AutoCorrelationLayer(
                AutoCorrelation(True, factor, attention_dropout=dropout,
                                output_attention=False),
                d_model, n_heads),
            AutoCorrelationLayer(
                AutoCorrelation(False, factor, attention_dropout=dropout,
                                output_attention=False),
                d_model, n_heads),
            d_model,
            c_out,
            d_ff,
            moving_avg=moving_avg,
            dropout=dropout,
            activation=activation,
        )
        for l in range(d_layers)
    ],
    norm_layer=my_Layernorm(d_model),
    projection=nn.Linear(d_model, c_out, bias=True)
).to(device)

In [22]:
seasonal_part, trend_part = decoder(dec_out, enc_out, x_mask=None, cross_mask=None,
                                            trend=trend_init)

In [23]:
seasonal_part.shape

torch.Size([32, 96, 1])

In [24]:
trend_part.shape

torch.Size([32, 96, 1])

In [25]:
# final
dec_out = trend_part + seasonal_part

dec_out.shape

torch.Size([32, 96, 1])

## visualization 

In [26]:
dec_out_np = dec_out.detach().cpu().numpy()
batch_y_np = batch_y.detach().cpu().numpy()
batch_x_np = batch_x.detach().cpu().numpy()
seasonal_np =seasonal_part.detach().cpu().numpy()
trend_np = trend_part.detach().cpu().numpy()

pred = dec_out_np[:, -pred_len:, :]
true = batch_y_np[:, -pred_len:, :]
input = batch_x_np
seasonal = seasonal_np 
trend = trend_np


In [41]:
import matplotlib.pyplot as plt  

plt.plot(seasonal[0].flatten())
plt.savefig('./test.png')
plt.show()

In [39]:
seasonal[0].flatten().shape

(96,)

In [42]:
plt.plot(pred[0].flatten())
plt.savefig('./test.png')